# <font color=red>Лекция 6.6</font> <font color=blue>Задание операций над множествами для соединений</font>

In [3]:
import pandas as pd
import numpy as np

  Во всех предыдущих лекциях мы игнорировали один важный нюанс выполнения соединения — вид используемой при соединении операции алгебры множеств. Это играет важную роль в случаях, когда какое-либо значение есть в одном ключевом столбце, но отсутствует в другом. Рассмотрим следующий пример:

In [7]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'], 'food': ['fish', 'beans', 'bread']},
                columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'], 'drink': ['wine', 'beer']},
                columns=['name', 'drink'])
print(df6)
print('===============')
print(df7)

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer


In [8]:
pd.merge(df6, df7)

,name,food,drink
0,Mary,bread,wine


В результате слито два набора данных, у которых совпадает только одна запись name: Mary. По умолчанию результат будет содержать пересечение двух входных множеств — внутреннее соединение (inner join). Можно указать это явным образом, с помощью ключевого слова how, имеющего по умолчанию значение 'inner':

In [5]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


Другие возможные значения ключевого слова how: 'outer', 'left' и'right'. Внешнее соединение (outer join) означает соединение по объединению входных столбцов и заполняет значениями NA все пропуски значений:

In [9]:
pd.merge(df6, df7, how='outer')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine
3,Joseph,NaN,beer


Левое соединение (left join) и правое соединение (right join) выполняют соединение по записям слева и справа соответственно. Например:

In [10]:
pd.merge(df6, df7, how='left')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine


Строки результата теперь соответствуют записям в левом из входных объектов. Опция `how='right'` работает аналогичным образом. Все эти опции можно непосредственно применять ко всем вышеописанным типам соединений.

### Пересекающиеся названия столбцов: ключевое слово suffixes
Может встретиться случай, когда в двух входных объектах присутствуют конфликтующие названия столбцов. Рассмотрим следующий пример:

In [11]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
print(df8)
print('===============')
print(df9);

   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2


In [12]:
pd.merge(df8, df9, on="name")

,name,rank_x,rank_y
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


Поскольку в результате должны были быть два конфликтующих названия столбцов, функция слияния автоматически добавила вназвания суффиксы _x и_y, чтобы обеспечить уникальность названий столбцов результата. Если подобное поведение, принятое по умолчанию, неуместно, можно задать пользовательские суффиксы с помощью ключевого слова suffixes:

In [13]:
pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])

,name,rank_L,rank_R
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


Эти суффиксы будут работать для всех возможных вариантов соединений, в том числе и в случае нескольких пересекающихся по названию столбцов. За более подробной информацией можно обратиться в раздел [Merge, Join and Concatenate "Слияние, соединение и конкатенация"](http://pandas.pydata.org/pandas-docs/stable/merging.html).

### <font color=red>Пример:</font> <font color=blue>Данные по штатам США</font>
Операции слияния и соединения чаще всего оказываются нужны при объединении данных из различных источников. Здесь мы рассмотрим пример с определенными данными по штатам США и их населению. Файлы данных можно найти по адресу http://github.com/jakevdp/data-USstates/:

In [16]:
pop = pd.read_csv('files/PandasFolder/state-population.csv')
areas = pd.read_csv('files/PandasFolder/state-areas.csv')
abbrevs = pd.read_csv('files/PandasFolder/state-abbrevs.csv')
print(pop.head())
print('===================================')
print(areas.head()) 
print('===================================')
print(abbrevs.head())

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


Допустим, нам нужно на основе этой информации отсортировать штаты и территорию США по плотности населения в 2010 году. Информации для этого у нас достаточно, но для достижения цели придется объединить наборы данных. Начнем со слияния «многие-ко-многим», которое позволит получить полное имя штата в объекте DataFrame для населения. Выполнить слияние нужно на основе столбца state/region объекта pop и столбца abbreviation объекта abbrevs. Воспользуемся опцией how='outer', чтобы гарантировать, что не упустим никаких данных из-за несовпадения меток.

In [17]:
merged = pd.merge(pop, abbrevs, how='outer', left_on='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', 1) # Удаляем дублирующуюся
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


Следует проверить, не было ли каких-то несовпадений. Сделать это можно путем поиска строк с пустыми значениями:

In [18]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

Часть информации о населении отсутствует, выясним, какая именно:

In [20]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


Похоже, что источник пустых значений по населению — Пуэрто-Рико, до 2000 года. Вероятно, это произошло из-за того, что необходимых данных не было в первоисточнике.

Мы видим, что некоторые из новых значений столбца state тоже пусты, а значит, в ключе объекта abbrevs отсутствовали соответствующие записи! 

Выясним, для каких территорий отсутствуют соответствующие значения:

In [21]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

Все понятно: наши данные по населению включают записи для Пуэрто-Рико (PR) и США в целом (USA), отсутствующие в ключе аббревиатур штатов. 

Исправим это, вставив соответствующие записи:

In [22]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

В столбце state больше нет пустых значений.

Теперь можно просмотреть результат с данными по площади штатов с помощью анало-гичной процедуры. После изучения имеющихся результатов становится понятно, что нужно выполнить соединение по столбцу state в обоих объектах:

In [23]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


Выполним снова проверку на пустые значения, чтобы узнать, были ли какие-то несовпадения:

In [24]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

В столбце area имеются пустые значения. Посмотрим, какие территории не были учтены:

In [26]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

Видим, что наш DataFrame-объект areas не содержит площадь США в целом.

Можно было бы вставить соответствующее значение (например, воспользовавшись суммой площадей всех штатов), но в данном случае мы просто удалим пустые значения, поскольку плотность населения США в целом нас сейчас не интересует:

In [27]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


Теперь у нас есть все необходимые нам данные. Чтобы ответить на интересующий вопрос, сначала выберем часть данных, соответствующих 2010 году и всему населению. Воспользуемся функцией `query()`:

In [33]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


Теперь вычислим плотность населения и выведем данные в соответствующем порядке. Начнем с переиндексации наших данных по штату, после чего вычислим результат:

In [34]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

Результат — список штатов США плюс Вашингтон (округ Колумбия) и Пуэрто-Рико, упорядоченный по плотности населения в 2010 году, в жителях на квадратную милю. 

Как видим, самая густонаселенная территория в этом наборе данных — Вашингтон (округ Колумбия); среди штатов же самый густонаселенный — Нью-Джерси. Можно также вывести окончание списка:

In [35]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

Как видим, штатом с наименьшей плотностью населения, причем с большим отрывом от остальных, оказалась Аляска, насчитывающая в среднем одного жителя на квадратную милю.

Подобное громоздкое слияние данных — распространенная задача при ответе на вопросы, связанные с реальными источниками данных.